In [58]:
import numpy as np
from scipy.optimize import root
from scipy.stats import norm
from scipy.stats import t
from scipy.optimize import minimize

# Entropic Risk Measure--  Liquidation Case

In [59]:
#####Before default####

mean_P = 2.0
var_P = 0.04    # 0.04
varrhos = np.ones(15)   #varrho_i for i=1,...,15
D = np.arange(len(varrhos)) + 1    #1,2,3,...,15
covs = np.sqrt(var_P) * ((-1)**(D+1) * 0.8) * (0.3*D) #cov_i = covs[i], for i=1,...,15

def Pre_default_position_liq_entropy_N(i):
    "Before defalt postions for i =1, ...15"
    varrho =  ((1/varrhos).sum())**(-1)
    cov = covs.sum()
    if  0<i and i<=15:
        return ((varrho/varrhos[i-1])*cov - covs[i-1])/var_P
    else:
        print('undefined')

def Pre_default_price_liq_entropy_N():
    "optimal price before default"
    varrho =  ((1/varrhos).sum())**(-1)
    cov = covs.sum()
    return mean_P - varrho * cov
    
######After Default#####

def Post_default_position_liq_entropy_N(i):
    "after defalt postions for i =1, ...14"
    varrho_prime = ((1/varrhos[0:14]).sum())**(-1)
    cov_prime = covs[0:14].sum()
    if 0<i and i<=14:
        return ((varrho_prime/varrhos[i-1])*cov_prime - covs[i-1])/var_P
    else:
        print ("undefined")    

def Post_default_price_liq_entropy_N():
    "optimal price fter default"
    varrho_prime = ((1/varrhos[0:14]).sum())**(-1)
    cov_prime = covs[0:14].sum()
    return mean_P -varrho_prime * cov_prime
                        
#Market Cost
def Market_cost_liq_entropy_N():
    varrho_prime = ((1/varrhos[0:14]).sum())**(-1)
    q_d = Pre_default_position_liq_entropy_N(15)
    return (0.5 * varrho_prime* (q_d**2) * var_P)

#def LC_liq_entropy_N():
 #   q_d = Pre_default_position_liq_entropy_N(15)
  #  P_prime =Post_default_price_liq_entropy_N()
   # P = Pre_default_price_liq_entropy_N()
    #return q_d*(P_prime -P)

# Risk Increment  Liquidation (Entropy, Normal)

$RI_i = r_i(q'_i) - r_i(q_i) + q_i'^\top p'-q_i^\top p$, 

where  $r_i(x) = -x\mu +\varrho_i xcov_i +\frac{1}{2}\varrho_ix^2\sigma^2$


In [60]:
def Risk_increment_liq_entropy_N():
    Pre = np.zeros(14)      #q_i, i=1,...,14
    Post = np.zeros(14)     #q_i, i=1,...14
    for i in range(0,14):
        Pre[i]=Pre_default_position_liq_entropy_N(i+1)
        Post[i]=Post_default_position_liq_entropy_N(i+1)
    Delta_q = Post-Pre
    P = Pre_default_price_liq_entropy_N()
    P_prime = Post_default_price_liq_entropy_N()
    varrhoss = varrhos[0:14]
    covss = covs[0:14]
    ter1 = (Post*P_prime) - (Pre*P)
    ter2 = (-(mean_P*Delta_q)) + (varrhoss*Delta_q*covss)
    ter3 = 0.5*var_P*varrhoss
    ter4 = (Post**2)-(Pre**2)
    return  ter1 + ter2 + (ter3*ter4)

def Price_impact_liq_entropy_N():
    Post = np.zeros(14)      #q_i, i=1,...,14
    for i in range(0,14):
        Post[i]=Post_default_position_liq_entropy_N(i+1)
    p = Pre_default_price_liq_entropy_N()
    P_prime = Post_default_price_liq_entropy_N()
    return (p-P_prime)*Post

In [61]:
Risk_increment_liq_entropy_N(), Price_impact_liq_entropy_N()

(array([-0.058528,  0.120032, -0.177568,  0.239072, -0.296608,  0.358112,
        -0.415648,  0.477152, -0.534688,  0.596192, -0.653728,  0.715232,
        -0.772768,  0.834272]),
 array([ 0.08928, -0.08928,  0.20832, -0.20832,  0.32736, -0.32736,
         0.4464 , -0.4464 ,  0.56544, -0.56544,  0.68448, -0.68448,
         0.80352, -0.80352]))

In [62]:
-0.031-Pre_default_position_liq_entropy_N(1)*(Pre_default_price_liq_entropy_N()-Post_default_price_liq_entropy_N())

-0.05877600000000005

In [63]:
#print the ppre and post default positions
DD = np.zeros(15)
DDD = np.zeros(14)
for i in range(0,15):
    DD[i]=Pre_default_position_liq_entropy_N(i+1)
for i in range(0,14):
    DDD[i]=Post_default_position_liq_entropy_N(i+1)
covs, DD, DDD  

(array([ 0.048, -0.096,  0.144, -0.192,  0.24 , -0.288,  0.336, -0.384,
         0.432, -0.48 ,  0.528, -0.576,  0.624, -0.672,  0.72 ]),
 array([ -0.56,   3.04,  -2.96,   5.44,  -5.36,   7.84,  -7.76,  10.24,
        -10.16,  12.64, -12.56,  15.04, -14.96,  17.44, -17.36]),
 array([ -1.8,   1.8,  -4.2,   4.2,  -6.6,   6.6,  -9. ,   9. , -11.4,
         11.4, -13.8,  13.8, -16.2,  16.2]))

In [64]:
Risk_increment_liq_entropy_N()[0]+ D[0]*(Pre_default_price_liq_entropy_N()-Post_default_price_liq_entropy_N())

-0.10812800000000033

In [69]:
Market_cost_liq_entropy_N()

0.43052800000000024

# Entropic Risk Measure--  Hedging Case

In [70]:
#Pre-default position and price ---- see the liquidation case
var_P = 0.04    # 0.04
mean_P = 2.0
varrhos = np.ones(15)   #varrho_i for i=1,...,15
D = np.arange(len(varrhos)) + 1    #1,2,3,...,15
covs = np.sqrt(var_P) * ((-1)**(D+1) * 0.8) * (0.3*D) #cov_i = covs[i], for i=1,...,15

varrho_c = 1.0

def Post_default_position_hedg_entropy_N(i):
    "after defalt postions for i =1, ...14, c=15"
    varrho_prime = ((1/varrhos[0:14]).sum() + (1/varrho_c) )**(-1)
    q_d  = Pre_default_position_liq_entropy_N(15)
    cov_prime = covs[0:14].sum() + (q_d*var_P)
    
    if 0<i and i<=14:
        return ((varrho_prime*cov_prime/(varrhos[i-1])) - (covs[i-1]))/var_P - (varrho_prime*q_d/(varrhos[i-1]))
    elif i==15:
        return (varrho_prime*cov_prime/varrho_c)/var_P - q_d - (varrho_prime*q_d/varrho_c)
    else:
        print ("undefined")    

def Post_default_price_hedg_entropy_N():
    "optimal price after default"
    varrho_prime = ((1/varrhos[0:14]).sum() + (1/varrho_c) )**(-1)
    q_d  = Pre_default_position_liq_entropy_N(15)
    cov_prime = covs[0:14].sum() + (q_d*var_P)
    return mean_P - varrho_prime * cov_prime + varrho_prime * var_P *q_d


#Market Cost
def Market_cost_hedging_entropy_N(): #this is based on the simplified expression
    varrho =  ((1/varrhos).sum())**(-1)
    cov = covs.sum()
    varrho_prime = ((1/varrhos[0:14]).sum() + (1/varrho_c) )**(-1)
    q_d  = Pre_default_position_liq_entropy_N(15)
    a = (varrho**2)* (varrho_prime - varrho_c)/(2*(varrho_c**2))
    b = (cov**2)/var_P 
    c = 0.5*varrho_prime * var_P *(q_d**2)
    d = varrho_prime *varrho *q_d * cov /varrho_c
    return (a *b) + c-d

def Market_cost_hedging_correct_entropy_N(): #this is based on {eq:hcnormalentro} 36
    Post_default_position =np.zeros(15)
    for i in range(0,15):
        Post_default_position[i] = Post_default_position_hedg_entropy_N(i+1)
    Pre_default_position = np.zeros(15)
    covss = np.zeros(15)
    varrhoss = np.zeros(15)
    for i in range(0,14):
        Pre_default_position[i] =Pre_default_position_liq_entropy_N(i+1)
        covss[i] = covs[i]
        varrhoss[i]=varrhos[i]
    q_d =  Pre_default_position_liq_entropy_N(15)
    covss[14] = q_d*var_P
    varrhoss[14] = varrho_c
    Delta_q = Post_default_position - Pre_default_position
    P = Pre_default_price_liq_entropy_N()
    a = varrhoss * Delta_q
    b = covss + var_P*(Pre_default_position + (0.5 * Delta_q))
    c = 0.5 * var_P * varrho_c * (q_d**2) 
    d = q_d * (mean_P -P )
    return ((a*b).sum()) + (c - d ) 

def LC_hedg_entropy_N():
    q_d = Pre_default_position_liq_entropy_N(15)
    P_prime =Post_default_price_hedg_entropy_N()
    P = Pre_default_price_liq_entropy_N()
    return q_d*(P_prime -P)

# Risk Increment  Hedging (Entropy, Normal)

$RI_i = r_i(q'_i) - r_i(q_i) + q_i'^\top p'-q_i^\top p$

$RI_c = r_i(q'_c) +q'_cp'$

where  $r_i(x) = -x\mu +\varrho_i xcov_i +\frac{1}{2}\varrho_ix^2\sigma^2$ for each surviving member.

$r_c(q'_c) = q_d(p-\mu) - q'_c\mu + 0.5 \varrho_c\sigma^2 q_d^2 +\varrho_c \sigma^2 q_d q'_c +0.5 \sigma^2 \varrho_c {q'_c}^2 $.
        


In [71]:
def Risk_increment_hedg_entropy_N():
    Pre = np.zeros(14)      #q_i, i=1,...,14
    Post = np.zeros(14)     #q_i, i=1,...14
    for i in range(0,14):
        Pre[i]=Pre_default_position_liq_entropy_N(i+1)
        Post[i]=Post_default_position_hedg_entropy_N(i+1)
    Delta_q = Post-Pre
    varrhoss = varrhos[0:14]
    covss = covs[0:14]
    p = Pre_default_price_liq_entropy_N()
    P_prime = Post_default_price_hedg_entropy_N()
    q_prime_c = Post_default_position_hedg_entropy_N(15)
    q_d = Pre_default_position_liq_entropy_N(15)  
    ter1 = (Post*P_prime) - (Pre*p)
    ter2 = (-(mean_P*Delta_q)) + (varrhoss*Delta_q*covss)
    ter3 = 0.5*var_P*varrhoss
    ter4 = (Post**2)-(Pre**2)
    ter5 = (q_d*(p-mean_P)) - (q_prime_c*mean_P) + (0.5*varrho_c*(q_d**2)*var_P) + (varrho_c*q_prime_c*var_P*q_d)+(0.5*varrho_c*(q_prime_c**2)*var_P) + (q_prime_c*P_prime)   
    return  np.append((ter1 + ter2 + (ter3*ter4)),ter5)

def Price_impact_hedg_entropy_N():
    Pre = np.zeros(14)      #q_i, i=1,...,14
    for i in range(0,14):
        Pre[i]=Pre_default_position_liq_entropy_N(i+1)
    p = Pre_default_price_liq_entropy_N()
    P_prime = Post_default_price_hedg_entropy_N()
    return (p-P_prime)*Pre

In [72]:
Risk_increment_hedg_entropy_N(),Risk_increment_liq_entropy_N(),Price_impact_hedg_entropy_N()

(array([-0.05568 ,  0.11712 , -0.17088 ,  0.23232 , -0.28608 ,  0.34752 ,
        -0.40128 ,  0.46272 , -0.51648 ,  0.57792 , -0.63168 ,  0.69312 ,
        -0.74688 ,  0.80832 ,  0.827008]),
 array([-0.058528,  0.120032, -0.177568,  0.239072, -0.296608,  0.358112,
        -0.415648,  0.477152, -0.534688,  0.596192, -0.653728,  0.715232,
        -0.772768,  0.834272]),
 array([ 0.02688, -0.14592,  0.14208, -0.26112,  0.25728, -0.37632,
         0.37248, -0.49152,  0.48768, -0.60672,  0.60288, -0.72192,
         0.71808, -0.83712]))

In [73]:
Market_cost_liq_entropy_N(), Market_cost_hedging_correct_entropy_N(), Market_cost_hedging_entropy_N(),LC_hedg_entropy_N()

(0.43052800000000024,
 0.4238080000000011,
 0.4238080000000003,
 -0.8332800000000049)

In [74]:
for i in range(1,16):
    print(i, Post_default_position_hedg_entropy_N(i))

1 -1.7600000000000002
2 1.8400000000000003
3 -4.16
4 4.24
5 -6.5600000000000005
6 6.640000000000001
7 -8.960000000000003
8 9.040000000000003
9 -11.360000000000001
10 11.440000000000001
11 -13.760000000000002
12 13.84
13 -16.160000000000004
14 16.240000000000002
15 16.800000000000004


In [75]:
Pre_default_price_liq_entropy_N(), Post_default_price_hedg_entropy_N()

(1.9744, 2.0224)

In [76]:
Pre_default_position_liq_entropy_N(15)*var_P

-0.6944000000000002

In [77]:
Market_cost_liq_entropy_N() - Pre_default_position_liq_entropy_N(15)*Pre_default_price_liq_entropy_N(), 
Market_cost_hedging_entropy_N()

0.4238080000000003

# EXpected Shortfall--Liquidation case 

In [78]:
################################# Liquidation case ####### for normal distribution
var_P = 0.04    # 0.04
var_Rs = 0.09*D**2   #Variance of R_i, i=1,2,...,15 
mean_P = 2.0
varrhos = np.ones(15)   #varrho_i for i=1,...,15
varrho_c = 1.0
D = np.arange(len(varrhos)) + 1    #1,2,3,...,15
covs = np.sqrt(var_P) * ((-1)**(D+1) * 0.8) * (0.3*D) #cov_i = covs[i], for i=1,...,15

alpha = 0.975


def Pre_default_position_liq_ES_N():
    def fun(x):
        a = var_P*x + covs
        b = np.sqrt( var_Rs + 2*x*covs + (x**2)*var_P )
        c = a/b 
        return [c[0]-c[1], c[0]-c[2], c[0]-c[3], c[0]-c[4],c[0]-c[5], c[0]-c[6], c[0]-c[7], c[0]-c[8], c[0]-c[9], 
                c[0]-c[10], c[0]-c[11], c[0]-c[12], c[0]-c[13], c[0]-c[14], x.sum()]
    result = root(fun, np.zeros(15), tol=1e-30)
    optpositions=result.x
    return  optpositions

def Pre_default_price_liq_ES_N(pre_default_position_liq_ES_N_array):
    aa = norm.pdf(norm.ppf(alpha)) / (1-alpha)
    bb = np.sqrt( var_Rs + 2*pre_default_position_liq_ES_N_array * covs + 
                 (pre_default_position_liq_ES_N_array ** 2 ) * var_P )
    cc = var_P * (pre_default_position_liq_ES_N_array) + covs
    return mean_P - (aa/(bb[0])) * (cc[0])
    
# After Default
def Post_default_position_liq_ES_N():
    def fun(x):
        a = var_P * x + (covs[0:14])
        b = np.sqrt( var_Rs[0:14] + 2*x*(covs[0:14]) + (x**2)*var_P )
        c = a/b 
        return [c[0]-c[1], c[0]-c[2], c[0]-c[3], c[0]-c[4],c[0]-c[5], c[0]-c[6], c[0]-c[7], c[0]-c[8], c[0]-c[9], 
                c[0]-c[10], c[0]-c[11], c[0]-c[12], c[0]-c[13], x.sum()]
    result = root(fun, np.zeros(14), tol=1e-30)
    optpositions=result.x
    return  optpositions

def Post_default_price_liq_ES_N(post_default_position_liq_ES_N_array):
    aa = norm.pdf(norm.ppf(alpha)) / (1-alpha)
    bb = np.sqrt( var_Rs[0:14] + 2 * post_default_position_liq_ES_N_array * (covs[0:14]) + 
                 (post_default_position_liq_ES_N_array ** 2) * var_P )
    cc = var_P * post_default_position_liq_ES_N_array + (covs[0:14])
    return mean_P - (aa/(bb[0])) * (cc[0])

def Market_Cost_liq_ES_N(pre_default_position_liq_ES_N_array,
                         post_default_position_liq_ES_N_array,
                         pre_default_price_liq_ES_N):
    k = pre_default_position_liq_ES_N_array[0:14]
    kk = post_default_position_liq_ES_N_array
    q_d = pre_default_position_liq_ES_N_array[14]
    P = pre_default_price_liq_ES_N
    a = norm.pdf(norm.ppf(alpha)) / (1-alpha)
    b = np.sqrt( var_Rs[0:14] + 2*k*(covs[0:14]) + (k**2)*var_P )
    bb = np.sqrt( var_Rs[0:14] + 2*kk*(covs[0:14]) + (kk**2)*var_P )
    term1 = q_d * (P - mean_P)
    term2 = (a*(bb-b)).sum()
    return term1 + term2

#def LC_liq_ES_N():
 #   q_d = Pre_default_position_liq_ES_N()[14]
  #  P_prime =Post_default_price_liq_ES_N()
   # P = Pre_default_price_liq_ES_N()
    #return q_d*(P_prime -P)

# Risk Increment  Liquidation (Expectedshortfall, Normal)

$RI_i = r_i(q'_i) - r_i(q_i) + q_i'^\top p'-q_i^\top p$
, 

where  $r_i(x) = -x\mu +\mathbb{ES}_\alpha \sqrt{ var(R_i) +2x.cov_i+x^2\sigma^2}$

In [79]:
def Risk_increment_liq_ES_N(pre_default_position_liq_ES_N_array,
                            post_default_position_liq_ES_N_array,
                            pre_default_price_liq_ES_N,
                            post_default_price_liq_ES_N):
    k = pre_default_position_liq_ES_N_array[0:14]
    kk = post_default_position_liq_ES_N_array
    q_d = pre_default_position_liq_ES_N_array[14]
    p = pre_default_price_liq_ES_N
    P_prime = post_default_price_liq_ES_N
    a = norm.pdf(norm.ppf(alpha)) / (1-alpha)
    b = np.sqrt( var_Rs[0:14] + 2*k*(covs[0:14]) + (k**2)*var_P )
    bb = np.sqrt( var_Rs[0:14] + 2*kk*(covs[0:14]) + (kk**2)*var_P )
    Delta_q = kk-k
    ter1 = (kk*P_prime) - (k*p)
    ter2 = (-(mean_P*Delta_q)) + (a*(bb-b))
    return ter1 + ter2 
  
def Price_impact_liq_ES_N(post_default_position_liq_ES_N_array, p, p_prime):
    return (p - p_prime) * post_default_position_liq_ES_N_array

In [80]:
pre_default_position_liq_ES_N_array = Pre_default_position_liq_ES_N()
pre_default_price_liq_ES_N = Pre_default_price_liq_ES_N(pre_default_position_liq_ES_N_array)
post_default_position_liq_ES_N_array = Post_default_position_liq_ES_N()
post_default_price_liq_ES_N = Post_default_price_liq_ES_N(post_default_position_liq_ES_N_array)

In [81]:
(Risk_increment_liq_ES_N(pre_default_position_liq_ES_N_array,
                        post_default_position_liq_ES_N_array,
                        pre_default_price_liq_ES_N,
                        post_default_price_liq_ES_N).sum(),
Price_impact_liq_ES_N(post_default_position_liq_ES_N_array, pre_default_price_liq_ES_N, post_default_price_liq_ES_N),
Market_Cost_liq_ES_N(pre_default_position_liq_ES_N_array,
                     post_default_position_liq_ES_N_array,
                     pre_default_price_liq_ES_N))

(0.6954815916598411,
 array([ 0.10597815, -0.18546176,  0.31793444, -0.37092352,  0.52989074,
        -0.55638527,  0.74184703, -0.74184703,  0.95380333, -0.92730879,
         1.16575962, -1.11277055,  1.37771591, -1.2982323 ]),
 0.6954815916598429)

In [82]:
Pre_default_position_liq_ES_N(), Post_default_position_liq_ES_N()

(array([ -1.12,   2.56,  -3.36,   5.12,  -5.6 ,   7.68,  -7.84,  10.24,
        -10.08,  12.8 , -12.32,  15.36, -14.56,  17.92, -16.8 ]),
 array([ -1.28,   2.24,  -3.84,   4.48,  -6.4 ,   6.72,  -8.96,   8.96,
        -11.52,  11.2 , -14.08,  13.44, -16.64,  15.68]))

In [83]:
Market_Cost_liq_ES_N(pre_default_position_liq_ES_N_array,
                     post_default_position_liq_ES_N_array,
                     pre_default_price_liq_ES_N)

0.6954815916598429

In [84]:
(pre_default_price_liq_ES_N, post_default_price_liq_ES_N)

(1.9586022862107235, 2.0413977137892765)

In [85]:
##############Student t
var_P = 0.04    # 0.04
var_Rs = 0.09*D**2   #Variance of R_i, i=1,2,...,15 
mean_P = 2.0
varrhos = np.ones(15)   #varrho_i for i=1,...,15
varrho_c = 1.0
D = np.arange(len(varrhos)) + 1    #1,2,3,...,15
covs = np.sqrt(var_P) * ((-1)**(D+1) * 0.8) * (0.3*D) #cov_i = covs[i], for i=1,...,15
alpha = 0.975
nu = 2.5

def Pre_default_price_liq_ES_t(pre_default_position_liq_ES_N_array):
    a = np.sqrt((nu-2)/(nu-1))
    aa = (t.pdf(t.ppf(alpha, nu), nu) * (nu + t.ppf(alpha, nu)**2)) / ((1-alpha)*(nu-1))
    bb = np.sqrt( var_Rs + 2 * pre_default_position_liq_ES_N_array*covs + (pre_default_position_liq_ES_N_array**2)*var_P )
    cc = var_P*pre_default_position_liq_ES_N_array + covs
    return mean_P - ((a*aa)/(bb[0])) * (cc[0])

def Post_default_price_liq_ES_t(post_default_position_liq_ES_N_array):
    a = np.sqrt((nu-2)/(nu-1))
    aa = (t.pdf(t.ppf(alpha, nu), nu) * (nu + t.ppf(alpha, nu)**2)) / ((1-alpha)*(nu-1))
    bb = np.sqrt( var_Rs[0:14] + 2*post_default_position_liq_ES_N_array*(covs[0:14]) + 
                 (post_default_position_liq_ES_N_array**2)*var_P )
    cc = var_P*post_default_position_liq_ES_N_array + (covs[0:14])
    return mean_P - ((a*aa)/(bb[0])) * (cc[0])

def Market_Cost_liq_ES_t(pre_default_position_liq_ES_N_array,
                         post_default_position_liq_ES_N_array,
                         pre_default_price_liq_ES_t):
    k = pre_default_position_liq_ES_N_array[0:14]
    kk = post_default_position_liq_ES_N_array
    term0 = pre_default_position_liq_ES_N_array[14]*pre_default_price_liq_ES_t #q_d.p'
    a = np.sqrt((nu-2)/(nu-1))
    aa = (t.pdf(t.ppf(alpha, nu), nu) * (nu + t.ppf(alpha, nu)**2)) / ((1-alpha)*(nu-1))
    b = np.sqrt( var_Rs[0:14] + 2*k*((covs[0:14])) + (k**2)*var_P )
    bb = np.sqrt( var_Rs[0:14] + 2*kk*((covs[0:14])) + (kk**2)*var_P )
    term1 = -mean_P*(kk-k)
    term2 = (a*aa)*(bb-b)
    return term0 + (term1.sum()) + (term2.sum())
#def LC_liq_ES_t():
 #   q_d = Pre_default_position_liq_ES_N()[14]
 #   P_prime =Post_default_price_liq_ES_t()
 #   P = Pre_default_price_liq_ES_t()
  #  return q_d*(P_prime -P)


In [86]:
pre_default_price_liq_ES_t = Pre_default_price_liq_ES_t(pre_default_position_liq_ES_N_array)
post_default_price_liq_ES_t = Post_default_price_liq_ES_t(post_default_position_liq_ES_N_array)
market_Cost_liq_ES_t = Market_Cost_liq_ES_t(pre_default_position_liq_ES_N_array,
                                            post_default_position_liq_ES_N_array,
                                            pre_default_price_liq_ES_t)
(pre_default_price_liq_ES_t, post_default_price_liq_ES_t, market_Cost_liq_ES_t)

(1.9365549991316375, 2.0634450008683625, 1.0658760145884865)

 # Expected shortfall Headging case
Let $E=\{1,\dots,15\}$, $d=\{15\}$, $N=\{\mathrm{c}\}$, $\varrho_i=1=\varrho_\mathrm{c}$, and  $(R_i, P) \sim \mathcal{N}_2(\mu_i, \Sigma_i)$ with $\mathrm{cov}_i =\sigma \rho_i  \sigma_{R_i}$, where $\sigma= 0.2$, $\rho_i=(-1)^{i+1} 0.8$ is the correlation coefficient between $R_i$ and $P$, and $\sigma_{R_i}=0.3i$, $i \in E$.
When the CCP hedges on $\mathcal{E}$, the corresponding pre- and post-default optimal positions are given by :

In [87]:
# Before default
var_P = 0.04    # 0.04
var_Rs = 0.09*D**2   #Variance of R_i, i=1,2,...,15 
mean_P = 2.0
varrhos = np.ones(15)   #varrho_i for i=1,...,15
varrho_c = 1.0
D = np.arange(len(varrhos)) + 1    #1,2,3,...,15
covs = np.sqrt(var_P) * ((-1)**(D+1) * 0.8) * (0.3*D) #cov_i = covs[i], for i=1,...,15
alpha = 0.975

def Pre_default_position_hedg_ES_N():
    """
    Compute the predefault optimal positions for member i = 1,...,15. It works both for liquidation
    and headging cases.
    """
    def fun(x):
        a = var_P * x + covs
        b = np.sqrt( var_Rs + 2*x*covs + (x**2)*var_P )
        c = a/b 
        return [c[0]-c[1], c[0]-c[2], c[0]-c[3], c[0]-c[4],c[0]-c[5], c[0]-c[6], c[0]-c[7], c[0]-c[8], c[0]-c[9], 
                c[0]-c[10], c[0]-c[11], c[0]-c[12], c[0]-c[13], c[0]-c[14], x.sum()]
    result = root(fun, np.zeros(15), tol=1e-30)
    optpositions=result.x
    return  optpositions

def Pre_default_price_hedg_ES_N(pre_default_position_hedg_ES_N_array):
    """usefull for liquidation and hedging cases"""
    a = norm.pdf(norm.ppf(alpha)) / (1-alpha)
    b = (covs[0] + var_P*pre_default_position_hedg_ES_N_array[0]) 
    c = np.sqrt( var_Rs[0] + 2*pre_default_position_hedg_ES_N_array[0]*covs[0] + 
                (pre_default_position_hedg_ES_N_array[0]**2)*var_P )
    return  mean_P - (a*b )/c

    
def convex_conjugate(f, domain, y, pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N):
    """
    Compute the convex conjugate of a function f on a given domain
    at a point y.
    
    Parameters:
    f (function): the function to compute the convex conjugate of
    domain (tuple): a tuple of two values representing the lower and
                    upper bounds of the domain
    y (float): the value at which to compute the convex conjugate
    
    Returns:
    float: the value of the convex conjugate of f at y
    """
    def g(x, pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N):
        return x * y - f(x, pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N)
    # Find the supremum of g on the domain using Nelder-Mead algorithm
    res = minimize(lambda x: -g(x, pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N), 
                   x0=[0], bounds=[domain], method='Nelder-Mead')
    return -res.fun


# Define the function r_c
def r_c(x, pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N):
    q_d = pre_default_position_hedg_ES_N_array[14]
    var_c = (q_d**2)*var_P
    p = pre_default_price_hedg_ES_N
    a =  q_d*(p - mean_P ) - x*mean_P 
    b = norm.pdf(norm.ppf(alpha)) / (1-alpha)
    cc = np.sqrt( var_c + (2*x*var_P *q_d) + (var_P*(x**2)))
    return  a + b*cc 


def Post_default_position_hedg_ES_N(pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N):
    def fun(x):
        a = var_P*x[:14] + covs[:14]
        b = np.sqrt( (var_Rs[:14]) + 2*(x[:14])*(covs[:14]) + ((x[:14])**2)*var_P )
        c = a/b 
        """ now lets define the post default price pprime"""
        aa = norm.pdf(norm.ppf(alpha)) / (1-alpha)
        pprime = mean_P -aa*(c[0])
        q_d = pre_default_position_hedg_ES_N_array[14]
        return [c[0]-c[1], c[0]-c[2], c[0]-c[3], c[0]-c[4],c[0]-c[5], c[0]-c[6], c[0]-c[7], c[0]-c[8], c[0]-c[9], 
                c[0]-c[10], c[0]-c[11], c[0]-c[12], c[0]-c[13], 
                r_c(x[14], pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N) + 
                (x[14])*pprime + convex_conjugate(r_c, (-np.infty, np.infty), y=-pprime,
                                                  pre_default_position_hedg_ES_N_array=pre_default_position_hedg_ES_N_array,
                                                  pre_default_price_hedg_ES_N=pre_default_price_hedg_ES_N), 
                x.sum() + q_d]
    result = root(fun, np.zeros(15), tol=1e-30)
    optpositions=result.x
    return  optpositions

def Post_default_price_hedg_ES_N(post_default_position_hedg_ES_N_array):
    a = var_P * (post_default_position_hedg_ES_N_array[0]) + covs[0]
    b = np.sqrt( var_Rs[0] + 2* (post_default_position_hedg_ES_N_array[0])* (covs[0]) + 
                ((post_default_position_hedg_ES_N_array[0])**2)*var_P )
    aa = norm.pdf(norm.ppf(alpha)) / (1-alpha)
    return   mean_P -aa*(a/b)

def Market_Cost_hedg_ES_N(pre_default_position_hedg_ES_N_array, post_default_position_hedg_ES_N_array):
    q_d = pre_default_position_hedg_ES_N_array[14]
    Pre = pre_default_position_hedg_ES_N_array[0:14] #pre-default survival
    Post = post_default_position_hedg_ES_N_array[0:14] #Post-default survival
    q_prime_c = post_default_position_hedg_ES_N_array[14] #post default position of ccp
    var_Rss = var_Rs[0:14]       #variance of R_i for survival
    covss = covs[0:14]       #cov_i for survival
    a = norm.pdf(norm.ppf(alpha)) / (1-alpha)
    b = np.sqrt( var_Rss + (2*Pre*covss) + (Pre**2)*var_P )
    bb = np.sqrt( var_Rss + (2*Post*covss) + (Post**2)*var_P )
    Deltaq = Post-Pre
    ter1 = -mean_P*Deltaq 
    ter2 = a*(bb-b)
    return (ter1.sum())  + (ter2.sum()) + r_c(q_prime_c, pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N)

def LC_hedg_ES_N(pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N, post_default_price_hedg_ES_N):
    q_d = pre_default_position_hedg_ES_N_array[14]
    P_prime = post_default_price_hedg_ES_N
    P = pre_default_price_hedg_ES_N
    return q_d*(P_prime -P)


# Risk Increment  Hedging (Expectedshortfall, Normal)

$RI_i = r_i(q'_i) - r_i(q_i) + q_i'^\top p'-q_i^\top p$, 

$RI_c = r_i(q'_c) +q'_cp'$

where  $r_i(x) = -x\mu +\mathbb{ES}_\alpha \sqrt{ var(R_i) +2x.cov_i+x^2\sigma^2}$ for surviviving and 
            
    
$r_c(q'_c )$ is already coded 

In [88]:
def Risk_increment_hedg_ES_N(pre_default_position_hedg_ES_N_array,
                             post_default_position_hedg_ES_N_array,
                             pre_default_price_hedg_ES_N,
                             post_default_price_hedg_ES_N):
    q_d = pre_default_position_hedg_ES_N_array[14]
    Pre = pre_default_position_hedg_ES_N_array[0:14] #pre-default survival
    Post = post_default_position_hedg_ES_N_array[0:14] #Post-default survival
    q_prime_c = post_default_position_hedg_ES_N_array[14] #post default position of ccp
    var_Rss = var_Rs[0:14]       #variance of R_i for survival
    covss = covs[0:14]       #cov_i for survival
    a = norm.pdf(norm.ppf(alpha)) / (1-alpha)
    b = np.sqrt( var_Rss + (2*Pre*covss) + (Pre**2)*var_P )
    bb = np.sqrt( var_Rss + (2*Post*covss) + (Post**2)*var_P )
    Delta_q = Post-Pre
    P = pre_default_price_hedg_ES_N
    P_prime = post_default_price_hedg_ES_N
    ter1 =(Post*P_prime) - (Pre*P)
    ter2 =  (-(mean_P*Delta_q)) + (a*(bb-b))
    ter3 = r_c(q_prime_c, pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N) + (q_prime_c*P_prime)
    return np.append((ter1 + ter2), ter3)

def Price_impact_hedg_ES_N(pre_default_position_hedg_ES_N_array,
                           pre_default_price_hedg_ES_N,
                           post_default_price_hedg_ES_N):
    Pre =pre_default_position_hedg_ES_N_array[0:14]
    p = pre_default_price_hedg_ES_N
    P_prime = post_default_price_hedg_ES_N
    return (p-P_prime)*Pre

In [89]:
pre_default_position_hedg_ES_N_array = Pre_default_position_hedg_ES_N()
pre_default_price_hedg_ES_N = Pre_default_price_hedg_ES_N(pre_default_position_hedg_ES_N_array)
post_default_position_hedg_ES_N_array = Post_default_position_hedg_ES_N(pre_default_position_hedg_ES_N_array,
                                                                        pre_default_price_hedg_ES_N)
post_default_price_hedg_ES_N = Post_default_price_hedg_ES_N(post_default_position_hedg_ES_N_array)

In [90]:
Risk_increment_hedg_ES_N(pre_default_position_hedg_ES_N_array,
                         post_default_position_hedg_ES_N_array,
                         pre_default_price_hedg_ES_N,
                         post_default_price_hedg_ES_N).sum() \
+ Price_impact_hedg_ES_N(pre_default_position_hedg_ES_N_array,
                       pre_default_price_hedg_ES_N,
                       post_default_price_hedg_ES_N).sum()

0.6954817133685665

In [91]:
Market_Cost_hedg_ES_N(pre_default_position_hedg_ES_N_array, post_default_position_hedg_ES_N_array)

0.6954817133685651

In [92]:
Market_Cost_liq_ES_N(pre_default_position_liq_ES_N_array, post_default_position_liq_ES_N_array, pre_default_price_liq_ES_N)

0.6954815916598429

In [93]:
(Risk_increment_liq_ES_N(pre_default_position_liq_ES_N_array,
                        post_default_position_liq_ES_N_array,
                        pre_default_price_liq_ES_N,
                        post_default_price_liq_ES_N),
Price_impact_liq_ES_N(post_default_position_liq_ES_N_array, pre_default_price_liq_ES_N, post_default_price_liq_ES_N))

(array([-0.09935451,  0.19870903, -0.29806354,  0.39741805, -0.49677257,
         0.59612708, -0.69548159,  0.7948361 , -0.89419062,  0.99354513,
        -1.09289964,  1.19225416, -1.29160867,  1.39096318]),
 array([ 0.10597815, -0.18546176,  0.31793444, -0.37092352,  0.52989074,
        -0.55638527,  0.74184703, -0.74184703,  0.95380333, -0.92730879,
         1.16575962, -1.11277055,  1.37771591, -1.2982323 ]))

In [94]:
(Risk_increment_hedg_ES_N(pre_default_position_hedg_ES_N_array,
                         post_default_position_hedg_ES_N_array,
                         pre_default_price_hedg_ES_N,
                         post_default_price_hedg_ES_N).sum(),
Price_impact_hedg_ES_N(pre_default_position_hedg_ES_N_array,
                       pre_default_price_hedg_ES_N,
                       post_default_price_hedg_ES_N).sum())

(2.0864448616944014, -1.390963148325835)

In [95]:
(Market_Cost_hedg_ES_N(pre_default_position_hedg_ES_N_array, post_default_position_hedg_ES_N_array),
LC_hedg_ES_N(pre_default_position_hedg_ES_N_array, pre_default_price_hedg_ES_N, post_default_price_hedg_ES_N))

(0.6954817133685651, -1.390963148325835)

In [96]:
(pre_default_price_hedg_ES_N, post_default_price_hedg_ES_N)

(1.9586022862107235, 2.041397711706309)

In [97]:
(pre_default_price_liq_ES_N, post_default_price_liq_ES_N)

(1.9586022862107235, 2.0413977137892765)

# Positive definiteness can be checked using:
    $$var_Rs > (covs**2 /var_P)$$

In [98]:
var_Rs > (covs**2 /var_P)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])